# Experiment Workbench

In [1]:
import os, sys, math, datetime
import pathlib
from pathlib import Path
import numpy as np
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from dotenv import load_dotenv
import boto3
import wandb
from wandb.keras import WandbCallback
import mlflow
import deeplake


d:\Miniconda\envs\tiny_cnn\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.0.13) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:

print("Tensorflow version " + tf.__version__)
AUTOTUNE = tf.data.AUTOTUNE

# Confirm that TensorFlow can access GPU
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Tensorflow version 2.10.0
Found GPU at: /device:GPU:0


In [3]:
# Start a Tensorboard session
%load_ext tensorboard

In [4]:
#os.environ['WANDB_NOTEBOOK_NAME'] = 'Experiment Workbench'

BATCH_SIZE = 128
EPOCHS = 100
#LOGGING_STEPS = 64
LR = 0.001

PROJECT = "Tiny CNN"
MODELNAME = "Simple_Net"
EXPERIMENT = "MNIST"
RUN_NAME = "Run_8"

logdir = os.path.join("logs", MODELNAME, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

root_logdir = os.getcwd()

# Preparing the data

In [5]:
def parse_tfr_element(element):
    #use the same structure as above; it's kinda an outline of the structure we now want to create
    data = {
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width':tf.io.FixedLenFeature([], tf.int64),
        'image/class/label':tf.io.FixedLenFeature([], tf.int64),
        'image/encoded' : tf.io.FixedLenFeature([], tf.string),
        }

    # feature_dict = {
    #     'image/height':
    #         dataset_utils.int64_feature(image_height),
    #     'image/width':
    #         dataset_utils.int64_feature(image_width),
    #     'image/filename':
    #         dataset_utils.bytes_feature(filename.encode('utf8')),
    #     'image/source_id':
    #         dataset_utils.bytes_feature(str(image_id).encode('utf8')),
    #     'image/key/sha256':
    #         dataset_utils.bytes_feature(key.encode('utf8')),
    #     'image/encoded':
    #         dataset_utils.bytes_feature(encoded_jpg),
    #     'image/format':
    #         dataset_utils.bytes_feature('jpeg'.encode('utf8')),
    #     'image/class/label':
    #         dataset_utils.int64_feature(annotations['label']),
    #     'image/object/bbox/xmin':
    #         dataset_utils.float_list_feature(xmin),
    #     'image/object/bbox/xmax':
    #         dataset_utils.float_list_feature(xmax),
    #     'image/object/bbox/ymin':
    #         dataset_utils.float_list_feature(ymin),
    #     'image/object/bbox/ymax':
    #         dataset_utils.float_list_feature(ymax),
    #     'image/object/area':
    #         dataset_utils.float_list_feature(area),
    # }
    


    content = tf.io.parse_single_example(element, data)
    
    height = content['image/height']
    width = content['image/width']
    label = content['image/class/label']
    raw_image = content['image/encoded']
    
    
    #get our 'feature'-- our image -- and reshape it appropriately
    feature = tf.io.parse_tensor(raw_image, out_type=tf.int16)
    feature = tf.reshape(feature, shape=[height,width,3])
    return (feature, label)

In [5]:
#ds_train = deeplake.load('hub://activeloop/cifar10-train')
ds_test = ds = deeplake.load('hub://activeloop/cifar10-test')

DatasetHandlerError: A Deep Lake dataset does not exist at the given path (hub://activeloop/cifar10-test). Check the path provided or in case you want to create a new dataset, use deeplake.empty().

In [31]:
# Load some data
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
#y_train = keras.utils.to_categorical(y_train, 10)
#y_test = keras.utils.to_categorical(y_test, 10)


In [ ]:

input_shape = x_train.shape[1:]
output_shape = (96,96)
classes = 10
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]


In [21]:
input_shape

(32, 32, 3)

In [ ]:
rescale = tf.keras.layers.Rescaling(1./255, offset= -1)
train_ds = train_ds.map(lambda x, y: (rescale(x), y))
val_ds   = val_ds.map(lambda x, y: (rescale(x), y))

In [32]:
def data_preprocessing(image, label):
    # this function can be extended if any pre-processing or augmentation is needed
    image = tf.image.resize(image, output_shape, method="bicubic")
    return (image, label)

In [33]:
input_shape[:2]

(32, 32)

In [34]:

# Create a tf.data pipeline of augmented images (and their labels)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#train_dataset = train_dataset.batch(BATCH_SIZE).map(lambda x, y: (data_augmentation(x), y))
train_dataset = train_dataset.batch(BATCH_SIZE).map(data_preprocessing).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
#train_dataset = train_dataset.batch(BATCH_SIZE).map(lambda x, y: (data_augmentation(x), y))
val_dataset = val_dataset.batch(BATCH_SIZE).map(data_preprocessing).prefetch(tf.data.AUTOTUNE)


# Prepare the Lemon Quality Dataset

In [6]:
data_dir = Path.cwd().parent.joinpath("lemon_dataset", "docs", "data")

In [7]:
input_shape = (96,96)


In [8]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  interpolation="bilinear",
  image_size=(input_shape),
  batch_size=BATCH_SIZE
  )

val_dataset = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  interpolation="bilinear",
  image_size=(input_shape),
  batch_size=BATCH_SIZE
  )

Found 2528 files belonging to 3 classes.
Using 2023 files for training.
Found 2528 files belonging to 3 classes.
Using 505 files for validation.


In [9]:
classes = len(train_dataset.class_names)
labels = train_dataset.class_names

In [10]:
rescale = tf.keras.layers.Rescaling(1./255, offset= -1)
train_dataset = train_dataset.map(lambda x, y: (rescale(x), y)).prefetch(tf.data.AUTOTUNE)
val_dataset   = val_dataset.map(lambda x, y: (rescale(x), y)).prefetch(tf.data.AUTOTUNE)

# Define the model

In [11]:
mobilenet = tf.keras.applications.mobilenet.MobileNet(
    input_shape=(96,96,3),
    alpha=0.25,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights= None, #'imagenet',
    input_tensor=None,
    pooling=None,
    classes=classes,
    classifier_activation='softmax',
    #**kwargs
)

In [14]:
def train_model():

        # solve issue from: https://github.com/wandb/wandb/issues/3536
        if len(wandb.patched["tensorboard"]) > 0:
                wandb.tensorboard.unpatch()
                
        # Configure Tensorboard root log directory to read the debugging information
        
        wandb.tensorboard.patch(root_logdir=root_logdir)
        # wandb.tensorboard.patch(root_logdir="wandb.run.dir")
        
        wandb.init(
                # Set the project where this run will be logged
                project=PROJECT, 
                # Track hyperparameters and run metadata
                #config={
                #"learning_rate": LR,
                #"epochs": EPOCHS,
                #},
                sync_tensorboard=True
                )


        config = wandb.config
        # Specify the configuration variables
        config.dropout = 0.2
        #config.hidden_layer_size = 128
        #config.layer_1_size  = 16
        #config.layer_2_size = 32
        config.learn_rate = LR
        #config.decay = 1e-6
        #config.momentum = 0.9
        config.epochs = EPOCHS
        config.classes = classes

        # enable Tensorflow Debugging
        #tf.debugging.experimental.enable_dump_debug_info("./logs/debug", 
        #        tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

        model = mobilenet
        #model = create_model(input_shape, classes)
        model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

        logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= wandb.run.dir, histogram_freq=10, profile_batch="10, 20")
        #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= logdir, histogram_freq=1)
        wandb_callback = WandbCallback(input_type="image", labels=labels, validation_data = val_dataset)

        early_stopping = EarlyStopping(monitor="val_accuracy", patience=50)

        checkpoint = ModelCheckpoint("my_tiny_model", save_weights_only=True)
        history = model.fit(train_dataset,
                epochs=config.epochs, 
                validation_data=val_dataset, 
                callbacks=[tensorboard_callback,
                #wandb_callback, 
                checkpoint, 
                early_stopping]
        )

        wandb.finish()
        return history, model


In [15]:
wandb.finish()
history, model = train_model()

Epoch 1/100
16/16 [==============================] - 12s 427ms/step - loss: 0.0612 - accuracy: 0.9792 - val_loss: 2.9970 - val_accuracy: 0.4376
Epoch 2/100
16/16 [==============================] - 9s 548ms/step - loss: 0.0778 - accuracy: 0.9718 - val_loss: 2.8734 - val_accuracy: 0.4376
Epoch 3/100
16/16 [==============================] - 3s 159ms/step - loss: 0.0501 - accuracy: 0.9783 - val_loss: 2.7869 - val_accuracy: 0.4376
Epoch 4/100
16/16 [==============================] - 4s 209ms/step - loss: 0.0393 - accuracy: 0.9842 - val_loss: 3.0146 - val_accuracy: 0.4376
Epoch 5/100
16/16 [==============================] - 5s 253ms/step - loss: 0.0283 - accuracy: 0.9876 - val_loss: 3.1259 - val_accuracy: 0.4376
Epoch 6/100
16/16 [==============================] - 4s 202ms/step - loss: 0.0231 - accuracy: 0.9926 - val_loss: 2.9520 - val_accuracy: 0.4376
Epoch 7/100
16/16 [==============================] - 3s 180ms/step - loss: 0.0346 - accuracy: 0.9876 - val_loss: 3.3203 - val_accuracy: 0.437

In [ ]:
def run_multiple_training_runs(epochs, lrs):
    for epoch in epochs:
        for lr in lrs:
            run_training_run(epoch, lr)

# Try different values for the learning rate
epochs = [100, 120, 140]
lrs = [0.1, 0.01, 0.001, 0.0001]
run_multiple_training_runs(epochs, lrs)